In [154]:
import random

class Strategy():
    '''A class containing the genome for a strategy, as well as its fitness.'''
    def __init__(self, length, actions):
        #every number will correspond to an action
        self.length = length
        self.actions = actions
        self.genome = [random.randint(0, self.actions-1) for i in range(self.length)]
        self.fitness = None
        
    def __repr__(self):
        return str(self.genome)

class ChildStrategy(Strategy):
    '''A child strategy will create a child of the 2 parent strategies, with some random genes'''
    def __init__(self, parent1, parent2):
        if not (isinstance(parent1, Strategy) and isinstance(parent2, Strategy)):
            raise TypeError('parents\' type must be Strategy()')
        if parent1.length != parent2.length:
            raise TypeError('parents must have the same number of situations')
        if parent1.actions != parent2.actions:
            raise TypeError('parents must have the same number of actions')
        self.actions = parent1.actions
        self.length = parent1.length
        self.fitness = None
        self.genome = []
        for i in range(self.length):
            prob = random.random()
            # you could change these parameters to change the proportions of genes contributed by each parent and also the proportion of random genes
            if prob < 0.45:
                self.genome.append(parent1.genome[i])
            elif prob < 0.90:
                self.genome.append(parent2.genome[i])
            else:
                self.genome.append(random.randint(0,self.actions))

In [155]:
import pandas as pd
import os

class CurrentState():
    '''A class containing all the relevant information about the current state of the simulation'''
    def __init__(self, strategy):
        # the following is a sloppy way of finding a a ticker whose file actually exists
        with open('backtesting_data/tickers.txt') as tickers_files:
            tickers = [i.strip() for i in tickers_files]
            found = False
            while not found:
                index = random.randint(0, len(tickers) - 1)
                self.ticker = tickers[index]
                if os.path.isfile('backtesting_data/data/' + self.ticker + '.csv'):
                    found = True
        # here all of the main attributes are initialised
        self.strategy = strategy
        self.df = pd.read_csv('backtesting_data/data/' + self.ticker + '.csv')
        self.day = random.randint(1, len(self.df) - 1001)
        self.date = self.df.at[self.day, 'Date']
        self.liquidity = 100.0
        self.shares = 0.0
        self.high_y = self.df.at[self.day-1, 'High']
        self.high_t = self.df.at[self.day, 'High']
        self.low_y = self.df.at[self.day-1, 'Low']
        self.low_t = self.df.at[self.day, 'Low']
        self.close_t = self.df.at[self.day, 'Close']
        self.open_t = self.df.at[self.day, 'Open']
        self.investment = self.shares * self.open_t
        self.total_assets = self.liquidity + self.investment
    
    def next_day(self):
        #all of the main attributes are updated to the next day
        self.day += 1
        self.date = self.df.at[self.day, 'Date']
        self.high_y = self.df.at[self.day-1, 'High']
        self.high_t = self.df.at[self.day, 'High']
        self.low_y = self.df.at[self.day-1, 'Low']
        self.low_t = self.df.at[self.day, 'Low']
        self.close_t = self.df.at[self.day, 'Close']
        self.open_t = self.df.at[self.day, 'Open']
        self.investment = self.shares * self.open_t
        self.total_assets = self.liquidity + self.investment
    
    # the folllowing functions implement all of the actions the investor can take
    def _buy_all(self):
        if self.close_t == 0:
            return 'buy nothing'
        newshares = self.liquidity / self.close_t
        self.shares += newshares
        self.liquidity = 0.0
        return f'bought {newshares} shares (all) at {self.close_t}'
    
    def _buy_half(self):
        if self.close_t == 0:
            return 'buy nothing'
        newshares = (self.liquidity / 2) / self.close_t
        self.shares += newshares
        self.liquidity /= 2
        return f'bought {newshares} shares (half) at {self.close_t}'
    
    def _hold(self):
        return 'nothing'
    
    def _sell_half(self):
        soldshares = self.shares/2
        self.liquidity += soldshares * self.close_t
        self.shares /= 2
        return f'sold {soldshares} shares (half) at {self.close_t}'
    
    def _sell_all(self):
        soldshares = self.shares
        self.liquidity += self.shares * self.close_t
        self.shares = 0.0
        return f'sold {soldshares} shares (all) at {self.close_t}'
    
   
    def _recognize_situation(self):
        # this function translates boolean values into a decimal number from 0 to 15
        # which corresponds to an index in the genome list
        situation = -1
        if self.liquidity > self.investment:
            situation += 8
        if self.high_y > self.high_t:
            situation += 4
        if self.low_y > self.low_t:
            situation += 2
        if self.close_t > self.open_t:
            situation += 1
        return situation
    
    
    def act(self):
        # this function translates the index to an action taken by the investor
        situation = self._recognize_situation()
        action = self.strategy.genome[situation]
        if action == 0:
            return self._buy_all()
        elif action == 1:
            return self._buy_half()
        elif action == 2:
            return self._hold()
        elif action == 3:
            return self._sell_half()
        elif action == 4:
            return self._sell_all()
            

def test_strategy(strategy):
    # this function tests the strategy for 1000 days
    state = CurrentState(strategy)
    for i in range(1000):
        state.act()
        state.next_day()
    return(state.total_assets)

strat = Strategy(16,5)
print(test_strategy(strat))

76.91154084905386


In [166]:
import numpy as np
import pickle

def save_object(obj):
    try:
        with open("data.pickle", "wb") as f:
            pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
    except Exception as ex:
        print("Error during pickling object (Possibly unsupported):", ex)

class Generation():
    def __init__(self, population, total_fitness, fitness_list):
        self.population = population
        self.total_fitness = total_fitness
        self.fitness_list = fitness_list
    
    def __repr__(self):
        return f'pop: {self.population}\nfitness: {self.total_fitness}\n'

generations = []

population = [Strategy(16,5) for i in range(100)]

for generation in range(300):
    for strategy in population:
        fitness_list = [test_strategy(strategy) for i in range(100)]
        strategy.fitness = sum(fitness_list) / 100
    
    generation_fitness = [strategy.fitness for strategy in population]
    total_fitness = sum(generation_fitness)
    strategy_probabilities = [strategy.fitness/total_fitness for strategy in population]
    
    generations.append(Generation(population, total_fitness, generation_fitness))
    
    nextgen = []
    for i in range(len(population)):
        parent1 = np.random.choice(population, p = strategy_probabilities)
        parent2 = np.random.choice(population, p = strategy_probabilities)
        nextgen.append(ChildStrategy(parent1,parent2))
    population = nextgen[:]
    print(f'generation {generation} done\naverage fitness: {total_fitness/100}')

save_object(generations)

generation 0 done
average fitness: 142.93621090551795
generation 1 done
average fitness: 147.46929020067518
generation 2 done
average fitness: 157.97490577465362
generation 3 done
average fitness: 152.47112162475938
generation 4 done
average fitness: 153.4582717824487
generation 5 done
average fitness: 153.3803280195749
generation 6 done
average fitness: 153.02007151876512
generation 7 done
average fitness: 155.8553685481197
generation 8 done
average fitness: 153.43299876407318
generation 9 done
average fitness: 157.33130657040573
generation 10 done
average fitness: 157.20482948335663
generation 11 done
average fitness: 155.6788025357666
generation 12 done
average fitness: 156.31193391330046
generation 13 done
average fitness: 157.26631315594565
generation 14 done
average fitness: 159.18320506269592
generation 15 done
average fitness: 169.03410260541267
generation 16 done
average fitness: 162.4397022499044
generation 17 done
average fitness: 162.82516168700622
generation 18 done
averag

KeyboardInterrupt: 

In [172]:
print(generations[75].fitness_list)
print(generations[75].population)


[191.71065265703638, 164.2533169581069, 128.05161632864133, 195.47292535377116, 129.57493217367164, 186.1444994963552, 143.11829001545237, 150.07766616554872, 100.0, 155.18119433455425, 179.25296950922694, 173.9220509773567, 183.0464869048132, 177.50875510842653, 179.21082915533782, 186.37190503428505, 119.87030184990172, 140.88612589834685, 151.2403228375999, 176.7964369803488, 164.43236143903624, 177.7741653127436, 175.04138622097497, 131.92198883791215, 289.36169804512514, 158.9091205875889, 165.65635106488023, 196.38591247784035, 198.30038171355724, 153.1330432326909, 140.81470517554638, 182.5935538220618, 126.11809584954139, 129.4513794943249, 151.9656045743361, 179.090406384637, 163.8213213846692, 177.5970833731531, 157.01258642124205, 233.51574004717946, 150.24687531769575, 153.74326414554955, 121.92565118157363, 160.78922307120993, 172.18247748264199, 207.57833928799687, 153.148412659265, 163.78082994341915, 141.6075196636501, 171.54221512263703, 130.26990217167014, 186.5632042

[191.71065265703638, 164.2533169581069, 128.05161632864133, 195.47292535377116, 129.57493217367164, 186.1444994963552, 143.11829001545237, 150.07766616554872, 100.0, 155.18119433455425, 179.25296950922694, 173.9220509773567, 183.0464869048132, 177.50875510842653, 179.21082915533782, 186.37190503428505, 119.87030184990172, 140.88612589834685, 151.2403228375999, 176.7964369803488, 164.43236143903624, 177.7741653127436, 175.04138622097497, 131.92198883791215, 289.36169804512514, 158.9091205875889, 165.65635106488023, 196.38591247784035, 198.30038171355724, 153.1330432326909, 140.81470517554638, 182.5935538220618, 126.11809584954139, 129.4513794943249, 151.9656045743361, 179.090406384637, 163.8213213846692, 177.5970833731531, 157.01258642124205, 233.51574004717946, 150.24687531769575, 153.74326414554955, 121.92565118157363, 160.78922307120993, 172.18247748264199, 207.57833928799687, 153.148412659265, 163.78082994341915, 141.6075196636501, 171.54221512263703, 130.26990217167014, 186.56320425349105, 151.97244599338572, 173.7506500967261, 213.27482934106874, 180.31963937728918, 184.02894946449774, 183.3997937617466, 100.0, 178.1076233436336, 146.35703177369928, 174.25502439090593, 180.67659657594305, 169.458314838073, 136.39981487129384, 166.37942328258242, 171.82175539203672, 172.49473206886069, 159.1077909034914, 153.8809229209455, 145.2409792733172, 141.20083377524958, 148.22055373261728, 169.13858586484648, 123.33890045693646, 129.38675648936157, 247.38240256130717, 177.1649575098423, 170.48329063837306, 194.65305676654353, 199.28190689963208, 112.31356222591383, 162.42207858961473, 161.66445290549316, 135.0183239095604, 133.96975053403824, 162.86678455232433, 167.13146093748514, 183.95715778046375, 147.75324494409756, 179.21782932826386, 154.73185227497532, 158.35397769363345, 124.25200563827309, 132.48534373239445, 166.6322369544674, 185.20418647559265, 190.83922959911223, 170.221332714875, 180.99153868481707]
[[1, 2, 0, 1, 3, 1, 0, 2, 3, 5, 4, 2, 0, 1, 0, 2], [0, 1, 0, 2, 0, 1, 1, 5, 5, 1, 5, 5, 2, 5, 0, 2], [3, 1, 4, 2, 3, 5, 1, 0, 2, 4, 1, 4, 1, 3, 2, 5], [1, 2, 4, 5, 5, 0, 3, 4, 1, 4, 0, 0, 5, 0, 4, 0], [0, 2, 0, 5, 0, 1, 5, 5, 3, 5, 1, 3, 4, 4, 0, 4], [5, 0, 5, 4, 1, 1, 0, 0, 0, 0, 1, 2, 4, 0, 4, 3], [0, 5, 0, 2, 4, 2, 2, 2, 0, 5, 3, 4, 1, 1, 2, 3], [2, 5, 2, 4, 1, 1, 1, 0, 2, 3, 1, 0, 1, 0, 0, 3], [1, 5, 0, 1, 3, 5, 1, 5, 3, 4, 2, 2, 4, 5, 5, 2], [2, 0, 5, 2, 1, 5, 0, 5, 1, 1, 3, 1, 4, 5, 4, 3], [5, 1, 0, 2, 1, 3, 2, 4, 0, 5, 0, 0, 0, 1, 0, 1], [1, 5, 1, 2, 2, 1, 1, 2, 1, 3, 1, 2, 4, 0, 2, 3], [2, 0, 4, 2, 0, 1, 1, 0, 5, 4, 4, 2, 0, 1, 2, 1], [0, 1, 2, 2, 1, 2, 1, 5, 4, 3, 5, 0, 0, 3, 2, 3], [1, 2, 2, 4, 1, 5, 1, 0, 1, 5, 1, 1, 2, 1, 2, 5], [0, 2, 1, 0, 3, 2, 0, 0, 1, 4, 4, 4, 0, 1, 0, 2], [1, 5, 5, 5, 0, 4, 5, 0, 3, 4, 4, 5, 2, 3, 0, 2], [5, 0, 4, 2, 3, 1, 4, 4, 1, 5, 4, 3, 1, 1, 0, 1], [2, 0, 5, 1, 1, 5, 4, 0, 0, 5, 2, 2, 1, 4, 4, 3], [5, 0, 3, 3, 1, 1, 1, 3, 0, 3, 1, 2, 1, 5, 4, 5], [5, 5, 4, 2, 4, 2, 1, 0, 2, 5, 1, 2, 5, 1, 5, 3], [1, 1, 4, 5, 1, 5, 1, 0, 5, 0, 5, 3, 0, 5, 2, 2], [2, 1, 0, 3, 0, 1, 1, 0, 0, 1, 5, 2, 3, 1, 5, 2], [5, 4, 0, 2, 1, 1, 4, 0, 0, 5, 3, 2, 4, 3, 3, 4], [2, 1, 2, 5, 1, 2, 2, 4, 3, 5, 0, 2, 0, 1, 5, 0], [5, 2, 0, 5, 4, 0, 4, 0, 1, 5, 3, 3, 1, 4, 0, 0], [5, 4, 0, 2, 1, 1, 4, 0, 1, 5, 2, 3, 4, 0, 4, 2], [5, 0, 1, 2, 1, 5, 1, 0, 1, 5, 3, 4, 1, 4, 3, 0], [5, 0, 4, 2, 5, 2, 0, 5, 1, 5, 1, 2, 4, 0, 2, 1], [5, 0, 1, 4, 5, 1, 1, 3, 1, 3, 4, 3, 1, 1, 0, 3], [5, 5, 0, 0, 4, 1, 0, 4, 2, 5, 1, 3, 3, 1, 2, 3], [0, 1, 0, 2, 0, 1, 1, 2, 5, 4, 4, 3, 0, 1, 2, 2], [2, 3, 4, 4, 3, 5, 3, 2, 4, 0, 0, 2, 5, 5, 2, 1], [4, 2, 2, 4, 2, 2, 2, 2, 0, 4, 4, 5, 2, 1, 2, 2], [5, 2, 0, 3, 0, 5, 4, 0, 0, 0, 4, 2, 2, 1, 2, 2], [5, 1, 5, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 4, 3], [5, 5, 3, 3, 3, 2, 0, 0, 0, 0, 5, 2, 5, 1, 5, 5], [5, 1, 0, 1, 3, 1, 1, 0, 3, 3, 2, 5, 4, 1, 5, 2], [1, 2, 4, 2, 0, 5, 1, 0, 3, 0, 1, 2, 5, 4, 2, 4], [1, 0, 4, 5, 0, 2, 1, 0, 0, 0, 5, 0, 5, 0, 5, 0], [2, 0, 4, 4, 5, 1, 4, 3, 1, 0, 3, 2, 4, 5, 0, 1], [4, 0, 2, 5, 1, 1, 2, 1, 0, 4, 4, 0, 4, 5, 3, 2], [3, 5, 5, 2, 3, 5, 4, 3, 1, 0, 4, 2, 5, 4, 5, 1], [2, 2, 4, 2, 0, 2, 1, 3, 5, 0, 5, 3, 3, 1, 4, 2], [1, 2, 2, 1, 2, 1, 2, 2, 3, 4, 5, 1, 2, 0, 0, 0], [2, 5, 5, 1, 0, 1, 4, 0, 3, 0, 0, 3, 1, 4, 2, 1], [2, 2, 4, 2, 0, 1, 1, 0, 3, 5, 1, 0, 5, 5, 4, 3], [2, 1, 0, 1, 0, 1, 2, 0, 2, 5, 1, 4, 1, 3, 0, 3], [2, 3, 4, 3, 1, 1, 2, 2, 1, 1, 3, 2, 5, 5, 4, 5], [5, 5, 5, 3, 0, 2, 3, 0, 0, 5, 3, 3, 1, 0, 2, 3], [4, 0, 3, 0, 1, 1, 0, 2, 4, 3, 1, 3, 0, 1, 4, 2], [0, 5, 0, 2, 0, 1, 2, 0, 5, 1, 5, 3, 2, 1, 5, 2], [1, 1, 4, 0, 0, 1, 0, 4, 3, 4, 1, 2, 5, 1, 5, 4], [5, 5, 0, 0, 1, 2, 2, 2, 0, 4, 2, 1, 1, 5, 5, 5], [5, 1, 4, 1, 1, 1, 2, 0, 2, 2, 2, 0, 1, 5, 0, 5], [0, 0, 3, 3, 1, 1, 1, 0, 0, 3, 1, 2, 4, 5, 0, 5], [0, 1, 2, 2, 4, 4, 5, 0, 0, 4, 4, 2, 4, 3, 3, 1], [1, 0, 2, 5, 2, 1, 3, 3, 1, 4, 0, 3, 3, 5, 4, 0], [1, 1, 4, 2, 1, 2, 0, 5, 3, 5, 4, 4, 4, 3, 4, 4], [5, 2, 4, 5, 1, 1, 2, 0, 4, 1, 1, 5, 2, 1, 5, 5], [4, 1, 4, 4, 5, 1, 2, 0, 5, 5, 5, 5, 4, 1, 2, 3], [1, 5, 2, 4, 5, 4, 1, 2, 0, 3, 0, 3, 3, 0, 0, 3], [5, 0, 0, 0, 1, 1, 1, 0, 1, 3, 4, 0, 4, 1, 2, 3], [2, 3, 3, 2, 0, 2, 1, 3, 5, 0, 4, 5, 3, 1, 3, 2], [0, 5, 1, 2, 3, 1, 1, 0, 2, 1, 1, 0, 1, 1, 2, 4], [5, 5, 1, 2, 2, 2, 1, 1, 3, 3, 5, 3, 1, 1, 0, 1], [0, 1, 3, 0, 1, 5, 1, 0, 4, 3, 5, 0, 2, 5, 0, 3], [2, 5, 3, 5, 2, 1, 1, 0, 4, 3, 2, 4, 5, 1, 5, 1], [2, 2, 1, 3, 2, 2, 0, 0, 3, 5, 4, 2, 1, 1, 2, 2], [0, 2, 4, 0, 3, 1, 4, 0, 3, 5, 2, 2, 2, 1, 2, 3], [1, 0, 0, 2, 0, 1, 4, 0, 5, 4, 5, 2, 2, 1, 5, 4], [4, 3, 4, 4, 0, 2, 1, 0, 5, 0, 4, 1, 3, 4, 0, 3], [5, 5, 0, 3, 3, 5, 2, 0, 4, 0, 2, 2, 4, 1, 5, 2], [5, 5, 4, 2, 1, 0, 2, 0, 0, 5, 2, 1, 1, 1, 2, 4], [4, 5, 4, 5, 1, 4, 2, 0, 5, 5, 1, 2, 2, 4, 0, 4], [3, 5, 5, 3, 3, 5, 3, 0, 3, 0, 4, 2, 1, 4, 2, 1], [1, 2, 0, 5, 2, 1, 1, 0, 1, 5, 2, 3, 5, 0, 4, 5], [2, 5, 4, 3, 1, 1, 3, 0, 3, 1, 1, 2, 4, 1, 2, 1], [5, 1, 4, 2, 2, 1, 2, 0, 2, 0, 2, 1, 4, 4, 4, 1], [5, 0, 4, 2, 5, 1, 1, 0, 0, 5, 4, 4, 1, 0, 4, 0], [2, 0, 4, 2, 2, 1, 2, 0, 5, 0, 1, 3, 3, 1, 3, 5], [3, 1, 2, 2, 1, 2, 1, 5, 4, 3, 5, 3, 4, 3, 0, 3], [5, 1, 4, 3, 4, 4, 2, 0, 1, 4, 4, 2, 0, 1, 5, 1], [2, 2, 0, 4, 0, 5, 5, 0, 4, 2, 1, 2, 1, 1, 1, 2], [5, 1, 4, 2, 4, 4, 0, 0, 1, 4, 4, 1, 1, 1, 2, 5], [4, 2, 2, 1, 2, 1, 0, 5, 3, 1, 2, 3, 4, 1, 5, 3], [0, 0, 1, 4, 2, 1, 2, 2, 3, 5, 3, 1, 2, 0, 0, 0], [0, 5, 0, 3, 3, 0, 5, 0, 3, 0, 2, 3, 5, 1, 2, 2], [0, 5, 4, 1, 3, 4, 2, 2, 0, 3, 1, 1, 5, 4, 0, 5], [2, 4, 4, 2, 1, 1, 2, 4, 0, 1, 0, 5, 1, 1, 5, 4], [5, 0, 0, 2, 0, 1, 1, 0, 1, 4, 5, 5, 1, 1, 4, 0], [2, 0, 0, 3, 0, 1, 3, 0, 0, 0, 5, 3, 1, 1, 5, 5], [2, 0, 4, 4, 0, 1, 1, 2, 5, 4, 4, 3, 0, 0, 3, 2], [4, 0, 3, 0, 0, 1, 4, 2, 3, 0, 5, 2, 3, 4, 5, 2], [5, 2, 2, 4, 2, 0, 4, 2, 3, 2, 4, 3, 4, 5, 0, 5], [2, 0, 0, 2, 3, 1, 3, 0, 5, 5, 5, 3, 5, 1, 5, 1], [5, 0, 4, 2, 2, 1, 1, 0, 3, 5, 1, 3, 3, 1, 0, 1], [0, 0, 0, 2, 1, 1, 3, 0, 5, 0, 4, 0, 4, 4, 0, 0], [1, 5, 4, 5, 3, 1, 0, 0, 3, 4, 0, 3, 5, 0, 4, 0], [1, 2, 5, 5, 5, 1, 1, 0, 1, 5, 1, 3, 5, 2, 2, 5]]

In [152]:
strat = Strategy(16,5)
strat.genome = [0, 1, 4, 0, 0, 0, 0, 2, 1, 3, 0, 1, 1, 4, 1, 1]
print(test_strategy(strat))

66.04305121928027


In [165]:
# the folowing is just a way to test if the child strategy works

a = Strategy(16, 5)
b = Strategy(16, 5)
print(a)
print(b)
print('\n')

for i in range (10):
    print(ChildStrategy(a,b))

[0, 1, 0, 1, 3, 0, 2, 3, 2, 3, 1, 4, 0, 4, 2, 0]
[4, 1, 1, 2, 3, 2, 4, 3, 2, 2, 2, 3, 1, 2, 2, 4]


[4, 1, 0, 2, 3, 0, 4, 3, 2, 3, 1, 4, 0, 2, 2, 3]
[0, 4, 1, 1, 3, 4, 4, 3, 2, 2, 2, 4, 1, 2, 2, 0]
[0, 1, 0, 1, 3, 2, 4, 3, 2, 2, 2, 3, 1, 2, 2, 0]
[2, 1, 0, 1, 2, 0, 5, 3, 2, 3, 1, 4, 0, 2, 2, 4]
[0, 1, 0, 2, 3, 2, 2, 3, 2, 3, 2, 4, 4, 2, 2, 4]
[4, 1, 1, 5, 1, 2, 2, 3, 2, 3, 2, 4, 1, 2, 2, 4]
[0, 1, 1, 1, 3, 0, 2, 3, 2, 3, 2, 3, 1, 2, 2, 4]
[0, 1, 1, 2, 3, 0, 2, 3, 2, 2, 1, 3, 1, 2, 2, 4]
[4, 1, 1, 1, 3, 0, 1, 3, 2, 2, 1, 3, 0, 2, 4, 4]
[4, 1, 0, 5, 3, 2, 2, 4, 2, 2, 2, 4, 1, 2, 2, 0]
